---
## Code runthrough / Documentation
#### Friday: 2024-03-08
---

## I will do a showcase of the following implementations of the week.

#### Classes

#### Factory Create

#### Singleton Database Connection

#### Tests


---

---
#### * Summary 

I've decided to create the code around a potential shopping website with 5 main database items. I've decided to not use subclasses for products as I want products to be generic enough to be any product. Identified by name, manufacturer, etc. I didn't see any advantage of subclasses in this scenario, and didn't know how to force it into the project.

* product
* login
* card
* user
* transaction

These are basic `Classes` with tablename and columns of types and names.
We will later create an item with the `Factory` and insert to a database with `Singleton`

My `Factory` is initialized with an `item_type` ex. `product` the factory will be made to create that type.
With the `create()` function we can give a dictionary or kwargs. The function does validation and checks for mandatory fields. Then the item is returned.
I have a tied `Validator` class just to test if it was possible, currently i just check if price is invalid for product. I'm not sure this is the best methodology.

I have made 2 different `Singleton`. One using imported SQLAlchemy for ease of use. And one with sqlite3 with a more manual implementation. The SQLAlchemy implementation seems a lot better and would prefer that in the future. But my manual implementation thought me a few things about SQL.

My `unittests` tests the `Factory` by itself, and then `Singleton` using the `Factory` to do database instructions.

---

---
#### * Classes 

The item classes all have the same basic structure of using SQLalchemys base class which allows for definition of MetaData in terms of a schema for table with columns and datatypes.

This allows me to make the table for database, and helps me insert correct fields into the columns.

Some fields are optional and can be initialised with None.

---

In [82]:
import uuid
from sqlalchemy import Column, Integer, String, Float, JSON
from sqlalchemy.dialects.postgresql import UUID
from sqlalchemy.orm import declarative_base

Base = declarative_base() #Base class from sqlalchemy

class Product(Base):
    __tablename__ = 'products'

    id = Column(Integer, primary_key=True)
    uuid = Column(UUID(as_uuid=True), default=uuid.uuid4, unique=True, nullable=False)
    manufacturer_id = Column(String)
    manufacturer= Column(String)
    name = Column(String)
    price = Column(Float)
    currency = Column(String)
    quantity = Column(Integer)
    weight = Column(Float)
    color = Column(String)
    release_year = Column(Integer)
    description = Column(String)
    category = Column(String)
    sub_category = Column(String)
    rating = Column(JSON) 
    technical_specs = Column(JSON)

    def __init__(self, 
                uuid,
                manufacturer_id, 
                manufacturer, 
                name, 
                price, 
                currency, 
                quantity = None, 
                weight = None, 
                color = None, 
                release_year = None, 
                description = None, 
                category = None, 
                sub_category = None, 
                rating = None, 
                tecnical_specs = None):
        
        self.uuid = uuid
        self.manufacturer_id = manufacturer_id
        self.manufacturer = manufacturer
        self.name = name
        self.price = price
        self.currency = currency
        self.quantity = quantity
        self.weight = weight
        self.color = color
        self.release_year = release_year
        self.description = description
        self.category = category
        self.sub_category = sub_category
        self.rating = rating
        self.tecnical_specs = tecnical_specs

---
#### * Factory 

The factory can be initialized with any type in the type_map and allows for easy implementation later.
All I have to do is make the item class and then add it here in the type_map.

I take a dict or kwargs and checks if all mandatory fields are correct, add UUID if mandatory and return the item.

I decided to try to make a Validator class which also uses a map to assign all the validator functions to the types.
This seemed to be a scaleable solution. But when thinking about validation of data with data from the database; like unique username. The idea got a bit more messy than first anticipated.

---

In [83]:
#Notebook import hack
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
code_dir = os.path.abspath(os.path.join('..', 'code'))
sys.path.append(code_dir)

#Normal imports
import uuid
import inspect
from items import *

class Validator:
    @staticmethod
    def validate_price(data_dict):
        price = data_dict.get("price")
        if price is None or not isinstance(price, (int, float)) or price <= 0:
            raise ValueError("Invalid or missing price for product")

class Factory:
    def __init__(self, item_type=None):
        self.item_type = item_type
        self.type_map = {
            "product": Product,
            "transaction": Transaction,
            "user": User,
            "login": Login,
            "card": Card,
        }
        self.validation_map = {
            "product": [Validator.validate_price],
            # Add validation functions for other types as needed
            # ex. validate password length etc
        }

    def create(self, data_dict=None, **kwargs):
        cls = self.type_map.get(self.item_type)
        if cls is None:
            raise ValueError(f"Invalid item type: {self.item_type}")

        if data_dict is None and not kwargs:
            raise ValueError("No arguments provided")
        
        if data_dict is None:
            data_dict = kwargs
        else:
            data_dict.update(kwargs)

        mandatory_fields = self.get_mandatory_fields(cls)

        # Special checks for certain types
        validation_funcs = self.validation_map.get(self.item_type, [])
        for func in validation_funcs:
            func(data_dict)

        # If "uuid" is a mandatory field and its not provided, generate a UUID
        if "uuid" in mandatory_fields and "uuid" not in data_dict:
            data_dict["uuid"] = self.handle_uuid()

        self.check_mandatory_fields(data_dict, mandatory_fields)

        item = cls(**data_dict)

        return item

    def get_mandatory_fields(self, cls):
        sig = inspect.signature(cls.__init__)
        return [name for name, param in sig.parameters.items() if param.default == inspect.Parameter.empty and name != 'self']

    def check_mandatory_fields(self, product_dict, fields_to_check):
        missing_fields = [field for field in fields_to_check if product_dict.get(field) is None]
        if missing_fields:
            raise ValueError(f"Mandatory fields cannot be empty: {', '.join(missing_fields)}")

    def handle_uuid(self):
        return uuid.uuid4()

---
#### * Singleton Database 
The important part about the Singleton is to return the same class that was initially created. We do this by overwriting the __new__ function and return an existing instance if it exists.

I initially wanted to make the Singleton Database Connection using only SQLAlchemy, but later decided to try and make the SQL functions myself in a seperate sqlite3 Singleton.

SQLAlchemy also allowed me to use the `declarative_base()` for easy meta data assignment. Which I also use for my sqlite3 Singleton.

The SQLAlchemy implementation is easy using the session and engine i can easily create a table `type(product).metadata.create_all(engine)` and add my item `session.add(product)` `session.commit()`

But with the sqlite3 implementation i had to write my own functions to handle this. Create a table `create_table_from_class(type(product))` and add item `insert_object(product)`

---

---
### SingletonDatabaseConnectSQLAlchemy
---

In [84]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

class SingletonDatabaseConnectSQLAlchemy:
    def __new__(cls, db_url=None):
        if not hasattr(cls, 'instance'):
            cls.instance = super(SingletonDatabaseConnectSQLAlchemy, cls).__new__(cls)
            cls.instance.engine = create_engine(db_url)
            cls.instance.Session = sessionmaker(bind=cls.instance.engine)
        return cls.instance

    def get_session(self):
        return self.Session()
    
    def get_engine(self):
        return self.engine

---
### SingletonDatabaseConnect (sqlite3)
---

In [85]:
#Notebook import hack
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
code_dir = os.path.abspath(os.path.join('..', 'code'))
sys.path.append(code_dir)

import sqlite3
from uuid import UUID
from typing import Optional
import json
from factories.factory import Factory

class SingletonDatabaseConnect:
    def __new__(cls, db_url=None):
        if not hasattr(cls, 'instance'):
            cls.instance = super(SingletonDatabaseConnect, cls).__new__(cls)
            cls.instance.db_url = db_url
            cls.instance.connection = sqlite3.connect(cls.instance.db_url)
        return cls.instance

    def get_session(self):
        return self.connection

    def get_cursor(self):
        return self.connection.cursor()
    
    def create_table_from_class(self, cls):
        cursor = self.get_cursor()
        table_name = cls.__tablename__
        annotations = cls.__table__.columns
        if not annotations:
            raise ValueError(f"Class {cls.__name__} doesn't have any columns")
        fields = ", ".join([f"{column.name} {self.get_sqlite_type(column.type)}" for column in annotations])
        cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({fields})")
        self.connection.commit()

    def insert_object(self, obj):
        cursor = self.get_cursor()
        table_name = getattr(obj.__class__, '__tablename__', obj.__class__.__name__.lower() + "s")
        fields = ", ".join([column.name for column in obj.__class__.__table__.columns])
        placeholders = ", ".join("?" * len(fields.split(', ')))
        values = [getattr(obj, name) for name in fields.split(', ')]

        # Get the next ID
        id_index = fields.split(', ').index('id')
        values[id_index] = self.get_next_id(table_name)

        # Convert unsupported types to strings
        values = [str(value) if not isinstance(value, (int, float, str, bytes, type(None))) else value for value in values]
        cursor.execute(f"INSERT INTO {table_name} ({fields}) VALUES ({placeholders})", tuple(values))
        self.connection.commit()
    
    def get_next_id(self, table_name):
        cursor = self.get_cursor()
        cursor.execute(f"SELECT MAX(id) FROM {table_name}")
        max_id = cursor.fetchone()[0]
        return max_id + 1 if max_id is not None else 0

    def get_object(self, cls, **kwargs):
        cursor = self.get_cursor()
        table_name = getattr(cls, '__tablename__', cls.__name__.lower() + "s")
        filters = " AND ".join([f"{key} = ?" for key in kwargs.keys()])
        values = tuple(str(value) for value in kwargs.values())
        cursor.execute(f"SELECT * FROM {table_name} WHERE {filters}", values)
        row = cursor.fetchone()
        if row is None:
            raise ValueError(f"No object of type {cls.__name__} found in the database")
        else:
            return {description[0]: value for description, value in zip(cursor.description, row)}  # Return a dictionary with the column names as keys
        
    def get_all_objects(self, cls):
        cursor = self.get_cursor()
        table_name = getattr(cls, '__tablename__', cls.__name__.lower() + "s")
        cursor.execute(f"SELECT * FROM {table_name}")
        while True:
            row = cursor.fetchone()
            if row is None:
                break
            yield {description[0]: value for description, value in zip(cursor.description, row)}  # Yield a dictionary with the column names as keys

    def get_sqlite_type(self, type_hint):
        if isinstance(type_hint, str):
            return "TEXT"
        elif isinstance(type_hint, int):
            return "INTEGER"
        elif isinstance(type_hint, float):
            return "REAL"
        elif isinstance(type_hint, dict):
            return "TEXT"  # Store dicts as JSON strings
        elif type_hint in Factory().type_map.values():  # Check if the type hint is one of the types in the Factory class
            return "TEXT"  # Store the object as a JSON string
        else:
            return str(type_hint).upper()

---
#### * Tests

I've divided the test into 3 parts.

* Testing the factory
* Testing Singleton sqlite3 with factory
* Testing Singleton sqlalchemy with factory

I used pythons `import unittest` with a custom test runner for my results and debugging.

The TestData class just generates and holds simple hardcoded data for testing. This could later implement random data or data from a database.
Because the current data is static it's impossible to test for unforseen inputs. Which means the testing is generally flawed in my case. But I have used it for general debugging but could be improved further.

---

In [86]:
#Notebook import hack
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
code_dir = os.path.abspath(os.path.join('..', 'code'))
sys.path.append(code_dir)

import unittest
from factories.factory import Factory
from singletonDatabaseConnect import SingletonDatabaseConnect
from singletonDatabaseConnectSQLAlchemy import SingletonDatabaseConnectSQLAlchemy
from sqlalchemy import inspect
#python -m unittest newTest.py

class TestData(unittest.TestCase):
    def __init__(self):
        loginFactory = Factory("login")
        productFactory = Factory("product")
        userFactory = Factory("user")
        cardFactory = Factory("card")
        self.product_data = {
            "currency": "USD",
            "manufacturer": "Apple",
            "manufacturer_id": "APPLE_123",
            "price": 1000,
            "name": "iPhone",
            "color": "black"
        }
        self.login_data = {
            "username": "user",
            "password": "pass"
        }
        login = loginFactory.create(self.login_data)
        self.user_data = {
            "name": "John",
            "age": 30,
            "email": "john@email.com",
            "address": "123 fake st",
            "shipping_address": "123 fake st",
            "phone": "123456789",
            "login": login,  
        }
        self.card_data = {
            "card_number": "123456789",
            "card_holder_name": "John",
            "expiry_date": "12/23",
            "cvv": "123"
        }
        user = userFactory.create(self.user_data)
        product = productFactory.create(self.product_data)
        card = cardFactory.create(self.card_data)
        self.transaction_data = {
            "user": user,
            "product": product,
            "card": card,
        }

---
#### * Tests

* Testing the factory

Here i will just show a few test cases for various things that got created under development. I found it to be a good idea to create a new test everytime a new feature was implemented. This way I made sure I didn't destroy old functionality as i developed my project.

---

In [87]:
import unittest

class TestFactory(unittest.TestCase):
    def setUp(self):
        self.test_data = TestData()
        self.product_data = self.test_data.product_data
        self.login_data = self.test_data.login_data
        self.user_data = self.test_data.user_data
        self.card_data = self.test_data.card_data
        self.transaction_data = self.test_data.transaction_data

    def test_create_product(self):
        print("\n(Testing product creation)")
        productFactory = Factory("product")
        product = productFactory.create(self.product_data)
        print("Factory created product:")
        print("Product attributes:")
        for attr, value in product.__dict__.items():
            print(f"{attr}: {value}")
        for key, value in self.product_data.items():
            self.assertEqual(getattr(product, key), value)

    def test_create_product_invalid_price(self):
        print("\n(Testing product creation with invalid price)")
        productFactory = Factory("product")
        self.product_data["price"] = -100
        try:
            productFactory.create(self.product_data)
        except ValueError as e:
            print(f"Correctly caught an error: {e}")

suite = unittest.TestLoader().loadTestsFromTestCase(TestFactory)
test_result = unittest.TextTestRunner().run(suite)

print("\nTest Results:")
print("Ran {} tests".format(test_result.testsRun))
print("(failures={})".format(len(test_result.failures)))
for t, f in test_result.failures:
    print("\nFailure: {} ({})".format(t, f))

..
----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK



(Testing product creation)
Factory created product:
Product attributes:
_sa_instance_state: <sqlalchemy.orm.state.InstanceState object at 0x00000170A5F047D0>
uuid: 9d5d58ca-76a7-4cf6-ab6f-d43ce0ba253c
manufacturer_id: APPLE_123
manufacturer: Apple
name: iPhone
price: 1000
currency: USD
quantity: None
weight: None
color: black
release_year: None
description: None
category: None
sub_category: None
rating: None
tecnical_specs: None

(Testing product creation with invalid price)
Correctly caught an error: Invalid or missing price for product

Test Results:
Ran 2 tests
(failures=0)


---
#### * Tests

* Testing Singleton sqlite3 with factory

---

In [90]:
import unittest

class TestSingletonDatabaseConnect(unittest.TestCase):
    def setUp(self):
        self.db_url = ":memory:"
        self.db = SingletonDatabaseConnect(self.db_url)
        self.test_data = TestData()
        self.product_data = self.test_data.product_data
        self.login_data = self.test_data.login_data
        self.user_data = self.test_data.user_data
        self.card_data = self.test_data.card_data
        self.transaction_data = self.test_data.transaction_data

    def test_connection_is_singleton(self):
        print("\n(Testing database connection is a singleton)")
        db1 = SingletonDatabaseConnect(self.db_url)
        db2 = SingletonDatabaseConnect(self.db_url)
        connection1 = db1.connection
        connection2 = db2.connection
        print(f"Connection 1: {connection1}")
        print(f"Connection 2: {connection2}")
        self.assertIs(connection1, connection2)

    def test_factory_product(self):
        print("\n(Testing product creation and database insertion)")
        factory = Factory("product")
        product = factory.create(self.product_data)

        print("Product attributes:")
        for attr, value in product.__dict__.items():
            print(f"{attr}: {value}")

        self.db.create_table_from_class(type(product))
        self.db.insert_object(product)

        result = self.db.get_object(type(product), uuid = product.uuid)
        print("\nDatabase result:")
        for key, value in result.items():
            print(f"{key}: {value}")

        self.assertEqual(result["uuid"], str(product.uuid))

suite = unittest.TestLoader().loadTestsFromTestCase(TestSingletonDatabaseConnect)
test_result = unittest.TextTestRunner().run(suite)

print("\nTest Results:")
print("Ran {} tests".format(test_result.testsRun))
print("(failures={})".format(len(test_result.failures)))
for t, f in test_result.failures:
    print("\nFailure: {} ({})".format(t, f))

..
----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK



(Testing database connection is a singleton)
Connection 1: <sqlite3.Connection object at 0x00000170A5E67B50>
Connection 2: <sqlite3.Connection object at 0x00000170A5E67B50>

(Testing product creation and database insertion)
Product attributes:
_sa_instance_state: <sqlalchemy.orm.state.InstanceState object at 0x00000170A5E733B0>
uuid: 03c3dbb2-407d-4531-ac01-d34728fe353c
manufacturer_id: APPLE_123
manufacturer: Apple
name: iPhone
price: 1000
currency: USD
quantity: None
weight: None
color: black
release_year: None
description: None
category: None
sub_category: None
rating: None
tecnical_specs: None

Database result:
id: 18
uuid: 03c3dbb2-407d-4531-ac01-d34728fe353c
manufacturer_id: APPLE_123
manufacturer: Apple
name: iPhone
price: 1000.0
currency: USD
quantity: None
weight: None
color: black
release_year: None
description: None
category: None
sub_category: None
rating: None
technical_specs: None

Test Results:
Ran 2 tests
(failures=0)


---
#### * Tests

* Testing Singleton SQLAlchemy with factory

---

In [91]:
class TestSingletonDatabaseConnectSQLAlchemy(unittest.TestCase):
    def setUp(self):
        self.db_url = "sqlite:///:memory:"
        self.db = SingletonDatabaseConnectSQLAlchemy(self.db_url)
        self.test_data = TestData()
        self.product_data = self.test_data.product_data
        self.login_data = self.test_data.login_data
        self.user_data = self.test_data.user_data
        self.card_data = self.test_data.card_data
        self.transaction_data = self.test_data.transaction_data

    def test_engine_is_singleton(self):
        print("\n(Testing database engine is a singleton)")
        db1 = SingletonDatabaseConnectSQLAlchemy(self.db_url)
        db2 = SingletonDatabaseConnectSQLAlchemy(self.db_url)
        engine1 = db1.get_engine()
        engine2 = db2.get_engine()
        print(f"Engine 1: {engine1}")
        print(f"Engine 2: {engine2}")
        self.assertIs(engine1, engine2)

    def test_factory_product(self):
        print("\n(Testing product creation and database insertion)")
        factory = Factory("product")
        product = factory.create(self.product_data)

        print("Product attributes:")
        for attr, value in product.__dict__.items():
            print(f"{attr}: {value}")

        session = self.db.get_session()
        engine = self.db.get_engine()

        type(product).metadata.create_all(engine) # Create table

        session.add(product)
        session.commit()

        result = session.query(type(product)).filter_by(uuid=product.uuid).first()
        print("\nDatabase result:")
        for attr, value in result.__dict__.items():
            if attr != '_sa_instance_state':
                print(f"{attr}: {value}")

        self.assertEqual(result.uuid, product.uuid)


suite = unittest.TestLoader().loadTestsFromTestCase(TestSingletonDatabaseConnect)
test_result = unittest.TextTestRunner().run(suite)

print("\nTest Results:")
print("Ran {} tests".format(test_result.testsRun))
print("(failures={})".format(len(test_result.failures)))
for t, f in test_result.failures:
    print("\nFailure: {} ({})".format(t, f))

..
----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK



(Testing database connection is a singleton)
Connection 1: <sqlite3.Connection object at 0x00000170A5E67B50>
Connection 2: <sqlite3.Connection object at 0x00000170A5E67B50>

(Testing product creation and database insertion)
Product attributes:
_sa_instance_state: <sqlalchemy.orm.state.InstanceState object at 0x00000170A5E71250>
uuid: 726f2397-a1a6-4bfb-b72c-8071021e6641
manufacturer_id: APPLE_123
manufacturer: Apple
name: iPhone
price: 1000
currency: USD
quantity: None
weight: None
color: black
release_year: None
description: None
category: None
sub_category: None
rating: None
tecnical_specs: None

Database result:
id: 19
uuid: 726f2397-a1a6-4bfb-b72c-8071021e6641
manufacturer_id: APPLE_123
manufacturer: Apple
name: iPhone
price: 1000.0
currency: USD
quantity: None
weight: None
color: black
release_year: None
description: None
category: None
sub_category: None
rating: None
technical_specs: None

Test Results:
Ran 2 tests
(failures=0)


---
#### * Tests

* All the tests run from imported newTest.py

---

In [94]:
import unittest
from newTest import TestFactory, TestSingletonDatabaseConnect, TestSingletonDatabaseConnectSQLAlchemy

suite_factory = unittest.TestLoader().loadTestsFromTestCase(TestFactory)
suite_singleton_db = unittest.TestLoader().loadTestsFromTestCase(TestSingletonDatabaseConnect)
suite_singleton_db_sqlalchemy = unittest.TestLoader().loadTestsFromTestCase(TestSingletonDatabaseConnectSQLAlchemy)

all_suites = unittest.TestSuite([suite_factory, suite_singleton_db, suite_singleton_db_sqlalchemy])

test_result = unittest.TextTestRunner().run(all_suites)

print("\nTest Results:")
print("Ran {} tests".format(test_result.testsRun))
print("(failures={})".format(len(test_result.failures)))
for t, f in test_result.failures:
    print("\nFailure: {} ({})".format(t, f))

...............................
----------------------------------------------------------------------
Ran 31 tests in 0.058s

OK



Test Results:
Ran 31 tests
(failures=0)


---
#### * Conclusion

I initially made a bad OOP code structure which delayed me a bit. I have since improved the coder and feels it's pretty scaleable. However I feel it's far from perfect and can see various issues, ex how I planned to validate the data in `Factory` had some future issues with validation from database data.

The SQLalchemy implementation seems robust, where as my sqlite3 implementation seems way too complicated and after writing most of the functions I forgot what I was doing. Which means the complexity is probably too high.

For the classes.
I probably should have made an easier example first, just doing as the assignment told me to. Make a base product and then make subclasses.
The complexity of having a transaction include other items such as product, user etc.. gave me some troubles, but was eventually fixed.

I feel I learned something new about OOP and how to handle database items.

---